## Egyptian Doctors dataset
For information about it, please check this <a href='https://kaggle.com/datasets/5f4cc145ba1ceea40f691eaac800f64b754fa8b93ed2fa34d3a6e6276d1f384c'> LINK HERE </a>

# Table of Contents
<ul>
<li><a href="#sql">Creating the SQLite database</a></li>  
<li><a href="#di">Data wrangling and cleaning</a></li>  
<li><a href="#eda">Final data look</a></li>


<a id='intro'></a>

In [2]:
# Import used libraries

import csv
import sqlite3

<a id='sql'></a>
## Creating the SQLite database

In [3]:
# connect to a new or existing database
conn = sqlite3.connect('Docs_database.db')

# create the games table
conn.execute('''
    CREATE TABLE doctors (
        Source TEXT NULL,
        web_scraper_order TEXT NULL,
        specialization TEXT NULL,
        fees TEXT NULL,
        avg_rate TEXT NULL,
        waiting_time TEXT NULL,
        clinic_location TEXT NULL,
        rate_count TEXT NULL,
        doctor_views TEXT NULL,
        pages TEXT NULL
    )
''')


In [ ]:
# read the columns header name
with open('/kaggle/input/egyptian-doctors/Egyptian Doctors Data.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        print(row.keys())
        break

In [ ]:
# read the data from the CSV file
with open('/kaggle/input/egyptian-doctors/Egyptian Doctors Data.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    
    # insert the data into the doctors table
    for row in reader:
        conn.execute('''
            INSERT INTO doctors (
            Source ,
            web_scraper_order ,
            specialization ,
            fees ,
            avg_rate ,
            waiting_time ,
            clinic_location ,
            rate_count ,
            doctor_views ,
            pages
            ) 
            VALUES (? ,? ,? ,? ,? ,? ,? ,? ,? ,? )
        ''', (
            row["\ufeffSource"],row["web_scraper_order"],
            row["specialization"],row["fees"],
            row["avg_rate"],row["waiting_time"],row["clinic_location"],
            row["rate_count"],row["doctor_views"],row["pages"]
        )) 


# save the changes to the database
conn.commit()

# close the connection to the database
conn.close()

<b>The database file has been created and saved. Now, we can connect to it using the %sql magic command.

In [ ]:
# Load SQL
%load_ext sql

# Connection of the database
%sql sqlite:///Docs_database.db

<a id='di'></a>
## Data wrangling and cleaning  

In [ ]:

%sql SELECT * FROM doctors LIMIT 5

<br>
<h3>Extract numbers from columns to be used in the analysis</h3>
<br>

In [ ]:
# Extract the cost and convert it to integer 'fees column'

%sql \
UPDATE doctors \
SET fees = SUBSTR(fees,1,instr(fees,'E')-1)

In [ ]:
# Extract the time and convert it to integer 'waiting_time column'

%sql \
UPDATE doctors \
SET waiting_time = SUBSTR(waiting_time,INSTR(waiting_time,':')+1,INSTR(waiting_time,'M')-1)

In [ ]:
# Extract the rate and convert it to integer 'rate_count column'

%sql \
UPDATE doctors \
SET rate_count = SUBSTR(rate_count,INSTR(rate_count,'m')+1,INSTR(rate_count,'V')-1)

In [ ]:
# Extract the Views and convert it to integer 'doctor_views column'

%sql \
UPDATE doctors \
SET doctor_views = REPLACE((SUBSTR(doctor_views,1,INSTR(doctor_views,'V')-1)), ',','')

In [ ]:
# Extract clinic location area 'clinic_location column'

%sql \
UPDATE doctors \
SET clinic_location = CASE WHEN INSTR(clinic_location,':') == 0 THEN clinic_location \
                            ELSE SUBSTR(clinic_location, 1,instr(clinic_location,':')-1) \
                            END

In [ ]:
# Replace null in clinic_location

%sql UPDATE doctors SET clinic_location = case when clinic_location = 'null' then 'unknown' else clinic_location end

In [ ]:
%sql \
UPDATE doctors \
SET clinic_location = TRIM(clinic_location)

<h3> We can't change columns type because SQLite doesn't support that </h3>
<br>
Read This:  <a href='https://stackoverflow.com/questions/2083543/modify-a-columns-type-in-sqlite3'> Click Here

In [ ]:
%sql UPDATE doctors SET fees = CAST(fees AS INTEGER)
%sql UPDATE doctors SET avg_rate = CAST(avg_rate AS INTEGER)
%sql UPDATE doctors SET waiting_time = CAST(waiting_time AS INTEGER)
%sql UPDATE doctors SET rate_count = CAST(rate_count AS INTEGER)
%sql UPDATE doctors SET doctor_views = CAST(doctor_views AS INTEGER)

In [ ]:
# Print tables info

%sql PRAGMA table_info(doctors);

<h3>Drop unwanted values, columns, duplicates from the table</h3>
<br>

In [ ]:
# Drop unwanted columns

%sql ALTER TABLE doctors DROP COLUMN pages
%sql ALTER TABLE doctors DROP COLUMN Source

In [ ]:
# Remove some empty rows

%sql DELETE from doctors WHERE specialization ='' or specialization = 'Laboratory'or specialization = 'Scan Center' or fees = 0

In [ ]:
# set outliers to 0

%sql UPDATE doctors SET waiting_time = 0 WHERE waiting_time = 35791394

In [ ]:
# Delete duplicate rows

%sql delete FROM doctors \
WHERE web_scraper_order NOT IN (\
  SELECT MIN(web_scraper_order) \
  FROM doctors \
  GROUP by specialization, clinic_location, avg_rate, doctor_views\
)

In [ ]:
# Replace zeros with average value in 'waiting_time' column

%sql UPDATE doctors AS t1\
        SET waiting_time = (\
              SELECT ROUND(AVG(t2.waiting_time)) \
              FROM doctors AS t2 \
                  WHERE t2.specialization = t1.specialization\
            ) WHERE cast(t1.waiting_time as integer) < 2;

<a id='eda'></a>
# Data final look

In [ ]:

%sql SELECT * FROM doctors LIMIT 5


In [ ]:
# Average per each specialization

%sql SELECT \
        distinct(specialization),\
        fees as AVG_fees,\
        avg_rate as AVG_avg_rate,\
        waiting_time as AVG_waiting_time,\
        rate_count as AVG_rate_count,\
        doctor_views as AVG_doctor_views \
    FROM doctors \
        group by specialization \
        order by cast(doctor_views as int)



<br> 
<b>1- Most of the values are normal </b><br>
<b>2- 'doctor_views' column has outliers but it's okay because it's no need to change it</b><br>
<b>3- we have 38 specialization, 1020 distinct row and 95 clinic location</b>
